Après moultes délibérations, voici le code du dernier df utilisé pour le système de recommandation :

In [73]:
import pandas as pd
import ast
import pickle

In [2]:
link1 = "./BD/tmdb_full.csv"
df_tmdb = pd.read_csv(link1, low_memory = False)

In [26]:
link2 = "./BD/df_t_akas_clean.csv"
df_t_akas = pd.read_csv(link2)

In [43]:
link3 = "./BD/df_t_principals_clean.csv"
df_t_principals = pd.read_csv(link3)

In [45]:
link4 = "./BD/df_basic_name_clean.csv"
df_basic_name_clean = pd.read_csv(link4)

Nettoyage du df_tmdb

In [ ]:
#L'énoncé nous demandant d'afficher l'affiche du film, on commence par supprimer les lignes qui ne renseignent pas le lien de l'affiche (45413/309572)
df_tmdb_clean = df_tmdb.dropna(subset=['poster_path'])

#Il reste 1 ligne qui renvoie à un film pour public adulte, restons courtois et supprimons la!
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['adult'] == False]

#D'après notre étude de marché nous allons nous limiter aux films Français et Anglais, en faisant le filtre au niveau de la langue
#pour garder les films francophones et anglophones plutôt que français et anglais.
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['original_language'] == 'fr') | (df_tmdb_clean['original_language'] == 'en')]
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['spoken_languages'].str.contains('fr')) | (df_tmdb_clean['spoken_languages'].str.contains('en'))]

#La colonne 'release_date' doit être au format datetime pour filtrer les films > 1960 (suppression de 27335 lignes sur 264158 (10%)) et les classer:
df_tmdb_clean["release_date"] = pd.to_datetime(df_tmdb_clean["release_date"])
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean["release_date"].dt.year > 1980]
df_tmdb_clean = df_tmdb_clean.sort_values('release_date')

#On garde les films avec un runtime > 40  et < 200 pour éviter les courts métrages ou les documentaires de 5h:
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['runtime'] > 40) & (df_tmdb_clean['runtime'] < 200)]

#117 films pas encore sortis, on peut les retirer: 
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['status'] == 'Released']

#Filtrage des lignes avec genre non renseigné
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['genres'].str.len() > 2]
 
#Conversion de la colonne 'genre' en liste
df_tmdb_clean['genres'] = df_tmdb_clean['genres'].apply(ast.literal_eval) 

#Suppression des lignes qui n'ont pas de notes ou qui ont 10/10 car non pertinent
df_tmdb_clean = df_tmdb_clean[(df_tmdb_clean['vote_average'] > 0) & (df_tmdb_clean['vote_average'] < 10)]

#On enlève les overviews manquant pour avoir une table propre:
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean['overview'].isna() == False]

#On va maintenant sélectionner et réaranger les colonnes pertinentes:
#INDEX on vire car on en fera un propre
#ADULT non pertinent car on ne les garde pas
#BACKDROP_PATH non pertinent car on peut avoir les posters
#Seulement 22316/236823 avec un BUDGET > 0 : colonne non pertinente pour ML mais on garde pour afficher si besoin
#genres on garde
#Homepage Non pertinent 
#id inutile car on garde seulement celui de imdb
#imdb_id très utile si besoin de certaines APIs
#original_language interessant on garde
#original_title cool pour le site et pour garder un df clair
#overview sympa pour le site
#popularity génial
#poster_path top pour le site
#production_countries trop complexe a utiliser, on vire
#release_date super pour site
#revenue : Seulement 14% > 0 : colonne non pertinente 
#runtime très utile!
#spoken_language compliqué à utiliser mais il a servi pour filtrer
#status inutile car tous pareils après filtre
#tagline quasi tous vides ou mal rensignés, à virer
#title inutile avec original title+ liste titres
#video inutile car booléen
#vote_average super !
#vote_count on garde pour donner du poids à la note
#pruduction companies inutiles


df_tmdb_clean = df_tmdb_clean[['imdb_id', 'original_title', 'release_date', 'runtime', 'genres', 'original_language', 'vote_average', 'vote_count', 'popularity', 'overview', 'poster_path', 'budget']]

#On ajoute le liste des titres de films possibles de title akas:
df_tmdb_clean = pd.merge(df_tmdb_clean, df_t_akas, how = 'left', left_on = 'imdb_id', right_on = 'titleId')
df_tmdb_clean = df_tmdb_clean.drop('titleId', axis = 1)

#On renomme les colonnes pour faire jÔli
df_tmdb_clean = df_tmdb_clean.rename(columns= {'imdb_id' : 'ID', 'original_title' : 'Titre', 'release_date' : 'Date de sortie', 'runtime' : 'Durée (minutes)', 'genres' : 'Genres', 'original_language' : 'Langue Originale', 'vote_average' : 'Note moyenne', 'vote_count' : 'Nb de votes', 'popularity' : 'Score de popularité', 'overview': 'Résumé', 'poster_path' : 'Lien du poster', 'budget' : 'Budget', 'title' : 'Titres possibles'})

#on supprime les doublons dans les listes:
df_tmdb_clean['Titres possibles'] = df_tmdb_clean['Titres possibles'].apply(ast.literal_eval)
df_tmdb_clean['Titres possibles'] = df_tmdb_clean['Titres possibles'].apply(lambda x : list(set(x)))

#Une centaine de liste de titres en NaN: On les drop pour avoir un df propre
df_tmdb_clean = df_tmdb_clean.dropna()

#On rajoute un colonne 'décénnie' pour le ML:
df_tmdb_clean['Décénnie'] = df_tmdb_clean['Date de sortie'].dt.year.apply(lambda x : (x//10)*10)  

#Dernier filtre des films qui n'ont pas assez de votes pour avoir une note pertinente:
df_tmdb_clean = df_tmdb_clean[df_tmdb_clean["Nb de votes"] > 20]

Ajout des colonnes acteurs et realisateur pour avoir un bô df final:

In [ ]:
#On filtre les categories du df_t_principals sur des nouveaux df pour un futur merge:
df_t_principals_act = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_real = df_t_principals[df_t_principals['category'] == 'director']

#Préparation nom des personnes en fonction de leur ID:
df_basic_name_noms = df_basic_name_clean[['nconst', 'primaryName']]

#incorporation des noms aux différents df_t_principals:
df_liste_act = pd.merge(df_t_principals_act, df_basic_name_noms, how = 'left', on = 'nconst')
df_liste_real = pd.merge(df_t_principals_real, df_basic_name_noms, how = 'left', on = 'nconst')



#Il existe des NaN qui poseront problème par la suite, on peut les dropper car ils représentent à peine 1% du total:
df_liste_act = df_liste_act.dropna()
df_liste_real = df_liste_real.dropna()

#On drop les colonnes maintenant inutiles 'nconst' et 'category' et on groupby sur les tconst pour le merge:
df_liste_act = df_liste_act.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)
df_liste_real = df_liste_real.drop(['nconst', 'category'], axis = 1).groupby('tconst').agg(list)

#Premier merge pour la colonne "Liste Acteurs":
df_final = pd.merge(df_tmdb_clean, df_liste_act, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Liste acteurs'})

#2e pour l'ajout de la colonne "Réalisateurs":
df_final = pd.merge(df_final, df_liste_real, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'primaryName' : 'Réalisateurs'})

#Il y a environ 10 000 lignnes NaN dans les nouvelles colonnes : on drop pour un df propre:
df_final = df_final.dropna()

#On resest_index
df_final = df_final.reset_index().drop('index', axis = 1)

#Petit export csv pour pas refaire le code à chaque fois et garder transformer automatiquement les colonnes de listes en listes
df_final.to_csv("df_final.csv", index = False)

On veut la listes des acteurs, réals, genres et décénnies sans doublons pour le streamlit

In [87]:
#On créé une liste vide:
acteurs = []
reals = []
genres = []

#Fonction pour un futur "apply" qui ajoute chaque liste du df à la liste vide:
def liste_acteurs(x):
    acteurs.extend(x)

def liste_reals(x):
    reals.extend(x)

def liste_genres(x):
    genres.extend(x)   

df_final['Liste acteurs'].apply(liste_acteurs)
df_final['Réalisateurs'].apply(liste_reals)
df_final['Genres'].apply(liste_genres)

#Pour supprimer les doublons on passe par un set:

acteurs = list(set(acteurs))
reals = list(set(reals))
genres = list(set(genres))
decennies = list(df_final['Décénnie'].unique())

In [88]:
with open('mes_listes.pkl', 'wb') as f:
    pickle.dump(acteurs, f)
    pickle.dump(reals, f)
    pickle.dump(genres, f)
    pickle.dump(decennies, f)

In [97]:
df_final[df_final['Titre'] == 'NXT TakeOver XXV']

,ID,Titre,Date de sortie,Durée (minutes),Genres,Langue Originale,Note moyenne,Nb de votes,Score de popularité,Résumé,Lien du poster,Budget,Titres possibles,Décénnie,Liste acteurs,Réalisateurs
41866,tt10374352,NXT TakeOver XXV,2019-06-01,158,"[Drama, Action, Thriller]",en,8.4,30,3.165,NXT TakeOver XXV is a professional wrestling s...,/a40PSKkeW6YAqYVKqqbW05jSoGk.jpg,0,[NXT TakeOver: XXV],2010,"[Johnny Gargano, Austin Jenkins, Shayna Andrea...","[Kevin Dunn, Paul Levesque]"


In [98]:
df_final[df_final['Nb de votes'] > 40]

,ID,Titre,Date de sortie,Durée (minutes),Genres,Langue Originale,Note moyenne,Nb de votes,Score de popularité,Résumé,Lien du poster,Budget,Titres possibles,Décénnie,Liste acteurs,Réalisateurs
31,tt0082812,Night School,1981-01-07,88,"[Horror, Mystery, Thriller]",en,5.800,53,5.284,A Boston police detective investigates a serie...,/pGFnK1Npjth1iYm3Kq2gqjC0Ceb.jpg,0,"[Terror en los ojos, Fryktens øyne, Oczy pełne...",1980,"[Leonard Mann, Rachel Ward, Drew Snyder, Josep...",[Ken Hughes]
32,tt0086262,Scream,1981-01-08,82,"[Horror, Mystery]",en,4.128,43,12.860,A group of friends on a rafting trip down a ri...,/fENI8R4NOIqBMEbx7TlQHRLLbgx.jpg,0,"[Scream, The Outing, Крик, Scream - Der Shock ...",1980,"[Pepper Martin, Hank Worden, Ethan Wayne, Ann ...",[Byron Quisenberry]
36,tt0081455,Scanners,1981-01-14,103,"[Science Fiction, Horror]",en,6.738,1001,11.191,After a man with extraordinary—and frightening...,/VTqLdveNXxGsIAZL5I4RliTTt7.jpg,4100000,"[掃描者大對決, 夺命凶灵, Скенери, Agyfürkészők, Tarayıcı...",1980,"[Jennifer O'Neill, Stephen Lack, Patrick McGoo...",[David Cronenberg]
40,tt0082533,The Howling,1981-01-21,91,[Horror],en,6.563,641,17.589,After a bizarre and near fatal encounter with ...,/dEFMku7o825YhyUEHSlkvLDz1Dt.jpg,1000000,"[Grito de Horror, 늑대 여인의 음모, Hurlements, El Au...",1980,"[Dee Wallace, Patrick Macnee, Dennis Dugan, Ch...",[Joe Dante]
43,tt0084090,Inseminoid,1981-01-23,93,"[Thriller, Horror, Science Fiction]",en,3.939,74,8.075,A crew of interplanetary archaeologists is thr...,/pHEkhpaiyVe7zrsWhEflWMNpXGB.jpg,2000000,"[Planeta do Medo, Engendro, Inseminóide - O Pl...",1980,"[Robin Clarke, Jennifer Ashley, Stephanie Beac...",[Norman J. Warren]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49706,tt23219684,Mighty Morphin Power Rangers: Once & Always,2023-04-19,59,"[Action, Adventure, Science Fiction, Family]",en,6.783,161,97.924,"After tragedy strikes, an unlikely young hero ...",/vc87upO8vcAGj9OmgH3AIz6ikKB.jpg,0,"[Power Rangers: Agora e Sempre, Mighty Morphin...",2020,"[Walter Jones, Steve Cardenas, Catherine Suthe...",[Charlie Haskell]
49715,tt5635026,Peter Pan & Wendy,2023-04-20,106,"[Family, Fantasy, Action, Adventure]",en,5.895,277,1358.468,"Wendy Darling, a young girl afraid to leave he...",/9NXAlFEE7WDssbXSMgdacsUD58Y.jpg,0,"[Питер Пэн и Венди, Пітер Пен і Венді, Peter P...",2020,"[Alexander Molony, Ever Anderson, Jude Law, Al...",[David Lowery]
49716,tt20115096,A Tourist's Guide to Love,2023-04-21,96,"[Romance, Comedy, Adventure]",en,6.543,69,161.027,"After an unexpected break up, a travel executi...",/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,0,"[Guida turistica per innamorarsi, Туристически...",2020,"[Rachael Leigh Cook, Scott Ly, Ben Feldman, Mi...",[Steven K. Tsuchida]
49744,tt27197387,AKA,2023-04-28,122,"[Thriller, Action, Crime]",fr,7.024,148,988.329,A steely special ops agent finds his morality ...,/3BSxAjiporlwQTWzaHZ9Yrl5C9D.jpg,0,"[Kimdim Ben?, AKA, Agente Infiltrado, Ταυτότητ...",2020,"[Alban Lenoir, Éric Cantona, Thibault de Monta...",[Morgan S. Dalibert]
